In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
max_iteration = 30000
weight_decay = 0.01
NUM_EMPLOYEES = 11000
NUM_VAL = 1000
Upsilon_1 = 0.01
Upsilon_2 = 0.01
NUM_FEATURES = 20
BANDWIDTH = 1

In [3]:
X = np.random.normal(loc=0.0, scale=10.0, size=(NUM_EMPLOYEES, NUM_FEATURES)) # the training set

survival_times = np.zeros(NUM_EMPLOYEES) # the surivial time of each employees
for i in range(NUM_EMPLOYEES):
    age = np.random.exponential(np.sum(Upsilon_1 * X[i,:]**2)+np.sum(Upsilon_2 * X[i,:]), size=1)
    survival_times[i] = np.ceil(age)
    
maximum_ob_time = int(max(survival_times))   

Y = np.ones((NUM_EMPLOYEES, maximum_ob_time)) # the lifetime matrix of all employees, if one employee leave at the time interval k, then from Y[i,k](inlcude)  all entries are -1
for i in range(NUM_EMPLOYEES):
    Y[i, int(survival_times[i])-1:] = -1
    
square = np.sum(X ** 2, axis=1)
column_vec = square[:, np.newaxis]
row_vec = square[np.newaxis, :]
Gaussian_Kernel = np.exp(-1 * (-2 * X.dot(X.T) + column_vec + row_vec) / (2 * BANDWIDTH ** 2)) # the Gaussian Kernel 
# Gaussian_Kernel = X.dot(X.T)

In [ ]:
alpha = np.zeros_like(Y_train)
max_iteration = 20000
for t in range(max_iteration):
    index = int(np.floor(np.random.rand()*NUM_EMPLOYEES)) # randomly select one employee
    haty_index = (Gaussian_Kernel[index,:][np.newaxis,:].dot(alpha * Y)).reshape(-1) # calculate the predicted y vector
    haty_index /= (t+1) * weight_decay

    ## update alpha

    mask = Y[index,:]*haty_index < 1
    alpha[index,] += mask

    if t%2000 == 0:
        print("the iteration is:", t)
        hatY = np.zeros_like(Y)
        hatY += Gaussian_Kernel.T.dot(alpha*Y)
        hatY /= (t+1) * weight_decay
        predict_age = np.zeros(NUM_EMPLOYEES)
        for i in range(NUM_EMPLOYEES):
            predict_age[i] = np.sum(hatY[i,:] > 0 )+1
        
        acc = np.sum(Y*hatY > 0)/(NUM_EMPLOYEES *maximum_ob_time)
        print("the accuracy is:", acc)
        
        useful_pair = 0
        denominator = NUM_EMPLOYEES*(NUM_EMPLOYEES-1)/2
        
        for i in range(NUM_EMPLOYEES):
            for j in np.arange(NUM_EMPLOYEES-(i+1))+(i+1):
                if (survival_times[i]-survival_times[j])*(predict_age[i]-predict_age[j])> 0: # useful pair
                    useful_pair += 1
                if  survival_times[i] == survival_times[j]: # pair contains two employees whose lifetime are identical
                    denominator -= 1
        
        c_index = useful_pair/ denominator
        print("the c-index is,",c_index)
        
print(np.sum(predict_age == survival_times)/NUM_EMPLOYEES)        
x_axis = np.arange(maximum_ob_time)+1
y_axis = np.zeros(maximum_ob_time)
y_predict_axis = np.zeros(maximum_ob_time)

for i in range(maximum_ob_time):
    y_axis[i] = np.sum(survival_times == x_axis[i])
    y_predict_axis[i] = np.sum(predict_age == x_axis[i])
print
plt.plot(x_axis, y_axis, x_axis,y_predict_axis)
